https://github.com/microsoft/recommenders/blob/main/examples/01_prepare_data/data_split.ipynb

In [70]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [71]:
transaction = pd.read_csv('book_transactions.csv', encoding='cp949')

In [72]:
data = transaction[['회원번호', '책제목']]
ids = data.groupby('회원번호').책제목.count().reset_index().query("책제목 > 5").회원번호.unique()
data = data.query('회원번호 in @ids')

id2user = dict(enumerate(data.회원번호.unique()))
user2id = {j:i for i, j in id2user.items()}
id2book = dict(enumerate(data.책제목.unique()))
book2id = {j:i for i, j in id2book.items()}
#id2cat = dict(enumerate(data.카테고리.unique()))
#cat2id = {j:i for i, j in id2cat.items()}

data.회원번호 = data.회원번호.map(lambda x: user2id[x])
data.책제목 = data.책제목.map(lambda x: book2id[x])
data = data.drop_duplicates()

In [73]:
#data_cat = transaction.groupby('책제목')['카테고리'].unique().map(lambda x: x[0]).reset_index()

In [74]:
#data_cat['책제목'] = data_cat.책제목.map(lambda x: book2id[x])
#data_cat['카테고리'] = data_cat.카테고리.map(lambda x: cat2id[x])

In [75]:
#data_cat = dict(data_cat.values)

In [76]:
data.to_csv('main_data_yes24.csv', index=False)

In [77]:
#data_cat = transaction[['회원번호', '카테고리']]
#data_cat.회원번호 = data_cat.회원번호.map(lambda x: user2id[x])
#data_cat.카테고리 = data_cat.카테고리.map(lambda x: cat2id[x])
#data_cat = data_cat.drop_duplicates()

In [78]:
data_tr = pd.read_csv('main_data_yes24.csv')

In [79]:
data_tr.shape

(203652, 2)

In [80]:
train_data = data_tr.copy()

In [81]:
items = data.groupby('회원번호')['책제목'].unique()
test_data = items.map(lambda x: x[-1]).reset_index().values
train_data = train_data.append(pd.DataFrame(test_data, columns=['회원번호', '책제목'])).drop_duplicates(keep=False)

In [82]:
train_data.to_csv('main_data_train_yes24.csv', index=False)

In [83]:
item_num = data_tr.책제목.nunique()
item_list = set(range(item_num))
def noninteracted(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, 99)
    result = x[-1].reshape(1).tolist() + sampled.tolist()
    return result
res = items.map(noninteracted)

In [84]:
np.save('main_data_test_yes24.npy', res.reset_index().values, allow_pickle=True)

In [85]:
test_data_raw = np.load('main_data_test_yes24.npy', allow_pickle=True)

In [86]:
test_data = []
for data in test_data_raw:
    u = data[0]
    i = data[1]
    for item in i:
        test_data.append([u, int(item)])

In [87]:
print(data_tr.회원번호.nunique(), data_tr.책제목.nunique())

12124 60210


ml-1m

In [56]:
column_names = ['MovieID', 'Title', 'Genres']
movies = pd.read_csv("./ml-1m/movies.dat", sep = "::", names = column_names, encoding='latin-1', engine='python')

column_names = ['User_ID', 'Gender', 'Age', 'Occupation', 'Zip-code']
users = pd.read_csv("./ml-1m/users.dat", sep = "::", names = column_names, engine='python')

column_names = ['User_ID', 'MovieID', 'Rating', 'Timestamp']
ratings = pd.read_csv("./ml-1m/ratings.dat", sep = "::", names = column_names, engine='python')
ratings.head()

,User_ID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [57]:
ratings = ratings.sort_values(['User_ID', 'Timestamp'])[['User_ID', 'MovieID']].reset_index(drop=True)

In [58]:
ratings['User_ID'].value_counts().describe()

count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
Name: User_ID, dtype: float64

In [59]:
ratings_copy = ratings.copy()

In [60]:
items = ratings.groupby('User_ID')['MovieID'].unique()
test_data = items.map(lambda x: x[-1]).reset_index().values
train_data = ratings_copy.append(pd.DataFrame(test_data, columns=['User_ID', 'MovieID'])).drop_duplicates(keep=False)

In [62]:
train_data

,User_ID,MovieID
0,1,3186
1,1,1270
2,1,1721
3,1,1022
4,1,2340
...,...,...
1000203,6040,232
1000204,6040,2917
1000205,6040,1921
1000206,6040,1784


In [61]:
test_data

array([[   1,   48],
       [   2, 1917],
       [   3, 2081],
       ...,
       [6038, 1183],
       [6039, 1254],
       [6040, 1221]], dtype=int64)

In [63]:
train_data.to_csv('main_data_train_ml.csv', index=False)

In [64]:
item_num = ratings_copy.MovieID.nunique()
item_list = set(range(item_num))
def noninteracted(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, 99)
    result = x[-1].reshape(1).tolist() + sampled.tolist()
    return result
res = items.map(noninteracted)

In [65]:
np.save('main_data_test_ml.npy', res.reset_index().values, allow_pickle=True)

In [69]:
ratings.User_ID.max(), ratings.MovieID.max()

(6040, 3952)